# Делаем с собственным менеджером памяти
## Менеджер памяти

Создаем структуру для хранения узла дерева. В ней будет число (ключ) и 2 ссылки на другие такие структуры (левый и правый сын). 

Создадим 3 массива:
1. Ключ - что содержательно хранится в элементе
2. Левый сын - числовые массив
3. Правый сын - числовой массив

Каждая ячейка памяти имеет индекс + в левом сыне (просто чтобы не заводить еще одно поле) содержит номер следующей за ней ячейки памяти, поэтому для начала записываем туда на 1 индекс больше. 

<img src="images/mem1.png" width="500">

После инициализации мы заполнили одну ячейку ключом, левым сыном и правым сыном, а потом перешли на ту ячейку, номер которой был в левом сыне заполненной на предыдущем шаге.

<img src="images/mem2.png" width="500">

Тоже самое для след. шагов: 

<img src="images/mem3.png" width="500">

Теперь попробуем освободить элемент (например, удалим нулевой элемент и теперь его нужно добавить в список свободных). Для этого добавим удаленный элемент в начало нашего списка, причем в левого сына мы указываем индекс первого свободного (не следующего уже) 

<img src="images/mem4.png" width="500">


In [11]:
def initmemory(size_of_table):
    '''
    size_of_table - максимальное число ячеек
    '''
    # memory - вся память (вся таблица на скрине)
    memory_table = []
    for i in range(size_of_table):
        # [ключ, левый сын, правый сын]
        # что в ключе и правом сыне - неважно - поставим туда пока 0
        memory_table.append([0, i+1, 0])
    # возвращаем список memory и первый свободный (сначала он 0) - memory structure
    return [memory_table, 0]

# создать новый нод
def newnode(memory_structure):
    memory_table, first_free = memory_structure
    # теперь на место первого свободного кладем тот, что был до этого в левом сыне предыдущего свободного
    memory_structure[1] = memory_table[first_free][1]
    return first_free 

# удаление
# index - тот, что удаляем
def delnode(memory_structure, index):
    memory_table, first_free = memory_structure
    # берем ноду с номером index, кладем на место левого сына первый свободный
    memory_table[index][1] = first_free
    # меняем первый свободный в самой структуре
    memory_structure[1] = index

In [12]:
memstruct = initmemory(4)
memstruct

[[[0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0]], 0]

In [13]:
newnode(memstruct)

0

In [14]:
memstruct

[[[0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0]], 1]

In [15]:
newnode(memstruct)

1

In [16]:
memstruct

[[[0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0]], 2]

In [17]:
delnode(memstruct, 0)

In [18]:
memstruct

[[[0, 2, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0]], 0]

## Бинарное дерево поиска

* У каждого узла ключ и два сына - левый и правый 
* В левом поддереве ключи меньше, а в правом - больше
* Если ключи поступают в случайном порядке, то глубина дерева будет $O(\log N)$

**Поиск по дереву:** 

<img src="images/TREE.png" width="500">

In [ ]:
# функция для нахождения индекса элемента x в дереве
def find(memory_structure, root, x):
    '''
    memory_structure - наша структура память (на 0 позиции стоит таблица, на 1 позиции первый свободный элемент)
    root - корень дерева
    x - число, которое ищем
    '''
    # получаем ключ из таблицы памяти с номером ноды root 
    key = memory_structure[0][root][0] # изначально это значение, которое лежит в корне 
                                       #(в дальнейшем - значение в узле)
    # если x совпал с ключом в корне, то возвращаем индекс root
    if x == key:
        return root
    # если x < значения в узле
    elif x < key:
        # получаем индекс левого соседа
        left = memory_structure[0][root][1]
        # если левого соседа нет, то и значения такого нет (возвращаем -1)
        if left == -1: 
            return -1 
        else: 
            return find(memory_structure, left, x) # рекурсивно вызываем ту же функцию, 
                                                   # но теперь корень это левая вершина
    elif x > key:
        # получаем индекс правого соседа
        right = memory_structure[0][root][2] 
        if right == -1:
            return -1 
        else: 
            return find(memory_structure, right, x)

`Сложность поиска` в худшем случае $O(N)$, но если дерево сбалансированно (то есть элементы влево и вправо добавляются случайно), то тогда $O(\log N)$

**Добавление элемента в дерево:**

<img src="images/add.png" width="500">

Тоже самое что с функцией поиска, только теперь когда мы не находим элемент, мы возвращаем не -1, а добавляем на это место элемент 

In [22]:
def create_and_fill_node(memory_structure, key):
    index = newnode(memory_structure)
    memory_structure[0][index][0] = key
    memory_structure[0][index][1] = -1
    memory_structure[0][index][2] = -1
    return index


def add(memory_structure, root, x):
    key = memory_structure[0][root][0]
    if x < key:
        left = memory_structure[0][root][1]
        if left == -1: 
            memory_structure[0][root][1] = create_and_fill_node(memory_structure, x)
        else: 
            add(memory_structure, left, x)
    elif x > key:
        right = memory_structure[0][root][2] 
        if right == -1:
            memory_structure[0][root][1] = create_and_fill_node(memory_structure, x)
        else: 
            add(memory_structure, right, x)

`Сложность добавления` в худшем случае $O(H)$, где $H$ - высота дерева. Также в худшем случае $H$ может быть равно $N$, в срднем $H = \log N$. В массиве добавления элемента в середину происходит за $O(N)$ в среднем

Пример создания дерева:

In [23]:
memstruct = initmemory(20)
root = create_and_fill_node(memstruct, 8)

add(memstruct, root, 10)
add(memstruct, root, 9)
add(memstruct, root, 14)
add(memstruct, root, 13)
add(memstruct, root, 3)
add(memstruct, root, 1)
add(memstruct, root, 6)
add(memstruct, root, 4)
add(memstruct, root, 7)

**Удаление элемента:**

<img src="images/del.png" width="500">

<img src="images/del2.png" width="500">

<img src="images/del3.png" width="500">

## Представление дерева в Python

$[key, [left], [right]]$

<img src="images/example_tree.png" width="200">

In [25]:
[5, [2, None, [3, None, None]], [7, None, [8, None, None]]]

[5, [2, None, [3, None, None]], [7, None, [8, None, None]]]

## Обход дерева:
* сначала влево до упора (печатаем)
* потом на один этап наверх обратно в себя (печатаем)
* потом вправо на один вниз (печатаем) 
* повторяем... 

<img src="images/path.png" width="500">

## Резюме:

* Дерево - отличная структура данных, когда у нас есть операции по поиску, добавлению и удалению элементов. 
* + в дереве можем искать следующий по величине элемент в отличие от хеш-таблицы. Однако, бин поиск тоже позволяет это делать и лучше использовать его по возможности

## Сериализация дерева Хаффмана

* Алгоритм Хаффмана позволяет сопоставить более часто встречающимся символам более короткий код
* Каждый раз берем два самых редко встречающихся символа и объединяем их в один узел 
* Строим бинарное дерево, кладем буквы в листья. Переход в левого сына кодируется числом 0, в правого числом 1, а код символа - это все ребра на пути от корня до листа
* Например, буква 'a' встречается 4 раза, 'б' - 3 раза, а 'в' и 'г' по одному разу. Тогда дерево выглядит так:

<img src="images/haff.png" width="200">

Тогда буквы кодируются следующим образом: а = 0, б = 11, в = 100, г = 101, причем коды получаются _безпрефиксные_ (коды не являются началом других), то есть нет например кода 10 и 100 - это круто, потому что иначе было бы непонятно, что брать, когда расшифровываешь последовательность 011100

## Как сохранить структуру дерева в виде строки? 

1. Обычный обход дерева:

L - в левого ребенка, R - в правого, U - в предка. Тогда для примера выше: LURLLURUURUU 

2. D - в наиболее левого непосещенного ребенка (детей всегда либо два либо 0), U - в предка

Тогда для примера выше: DUDDDUDUUDUU

3. D - в наиболее левого непосещенного ребенка (детей всегда либо два либо 0), U - поднимаемся вверх до тех пор, пока приходим из правого, а если стали из левого, то переходим в правого - за это все отвечает одна U 

Тогда для примера выше: DUDDUU

## Задача: 

> По сериализованной записи, составленной кодом из пункта 3 восстановить код для листьев

In [ ]:
def maketree(serialized):
    tree = {'left': None, 'right': None, 'up': None, 'type': 'root'}
    nownode = tree 
    for sym in serialized:
        if sym == 'D':
            newnode = {'left': None, 'right': None, 'up': nownode, 'type': 'left'}
            nownode['left'] = newnode
            nownode = newnode
        elif sym == 'U':
            while nownode['type'] == 'right':
                nownode = nownode['up']
            nownode = nownode['up']
            newnode = {'left': None, 'right': None, 'up': nownode, 'type': 'right'}
            nownode['right'] = newnode
            nownode = newnode
    return tree

def traverse(root, prefix):
    if root['left'] is None and root['right'] is None:
        return [''.join(prefix)]
    prefix.append('0')
    ans = traverse(root['left'], prefix)
    prefix.pop()
    prefix.append('1')
    
    ans.extend(traverse(root['right'], prefix))
    prefix.pop()
    return ans